In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from wandelbots import Instance, MotionGroup, Planner
from wandelbots.types import Pose

## ⚙️ Configure Robot

In [3]:
my_instance = Instance(
    url=os.getenv("WANDELAPI_BASE_URL"),
    user=os.getenv("NOVA_USERNAME"),
    password=os.getenv("NOVA_PASSWORD"),
)

my_robot = MotionGroup(
    instance=my_instance,
    cell=os.getenv("CELL_ID"),
    motion_group=os.getenv("MOTION_GROUP"),
    default_tcp=os.getenv("TCP"),
)

HOME = [0, -1.571, 1.571, -1.571, -1.571, 0]

## 🔩 Get Robot Props & State

In [4]:
print(my_robot.tcps())
print([round(j,2) for j in my_robot.current_joints()])
print(my_robot.current_tcp_pose())

['Flange']
[1.17, -1.66, 1.4, -1.57, -1.57, 1.17]
Position: [  -91,  -662,   851]
Rotation: [ 2.14,  2.14, -0.36]


## 📝 Perform Pose Transforms

In [5]:
current_pose = my_robot.current_tcp_pose()
translation = Pose.from_list([0, 0, -500, 0, 0, 0])
target_pose = translation * current_pose
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

Current
Position: [  -91,  -662,   851]
Rotation: [ 2.14,  2.14, -0.36]

Target
Position: [  -91,  -662,   351]
Rotation: [ 2.14,  2.14, -0.36]


## 🎢 Plan a Motion

In [6]:
planner = Planner(my_robot)
trajectory = [
    planner.jptp(joints=HOME),
    planner.line(pose=target_pose),
    planner.jptp(joints=HOME),
]
plan_result = planner.plan(
    start_joints=my_robot.current_joints(), trajectory=trajectory
)
print("motion id:", plan_result.motion)

motion id: ffb36a04-6ffc-451b-ae91-2984058a0c60


## 🏃🏽‍♀️ Execute Motion

In [7]:
async for state in my_robot.execute_motion_stream_async(motion=plan_result.motion, speed=25, response_rate_ms=500):
    current_location = state.current_location_on_trajectory
    print(f"Current Location: {current_location}")

Current Location: 0.05587199106230991
Current Location: 0.2165839189956829
Current Location: 0.43785051179364215
Current Location: 0.6595234836592838
Current Location: 0.8668485777731764
Current Location: 0.9812466221078786
Current Location: 1.0047674082229405
Current Location: 1.0808172926272794
Current Location: 1.2610340994405633
Current Location: 1.4808420828290316
Current Location: 1.670701217215598
Current Location: 1.8510355501595845
Current Location: 1.9704631684592295
Current Location: 2.0015819335854608
Current Location: 2.0683649178169827
Current Location: 2.2400071495017433
Current Location: 2.4616801206269328
Current Location: 2.68335309249254
Current Location: 2.8867778564245903
Current Location: 2.988390157900401
